In [1]:
import requests
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import pandas as pd
from random import randint
from time import sleep
import re
headers={'User-Agent': 'email@gmail.com'}

## Functions

In [2]:
def get_citations(content):
    out = 0
    for char in range(0,len(content)):
        if content[char:char+9] == 'Cited by ':
            init = char+9                          
            for end in range(init+1,init+6):
                if content[end] == '<':
                    break
            out = content[init:end]
    return int(out)

In [3]:
def get_year(content):
    for char in range(0,len(content)):
        if content[char] == '-':
            out = content[char-5:char-1]
    if not out.isdigit():
        out = 0
    return int(out)

In [4]:
def get_author(content):
    for char in range(0,len(content)):
        if content[char] == '-':
            out = content[2:char-1]
            break
    return out

In [5]:
def get_journal(content):
  out=re.findall("-(.*)-", content)[0]
  return out

In [24]:
def result(keyword, source, publisher, start_year, end_year, number_of_results):

  # Start new session
  session = requests.Session()

  # Variables
  links = list()
  title = list()
  citations = list()
  year = list()
  rank = list()
  author = list()
  journal=list()
  jour=list()
  rank.append(0) # initialization necessary for incremental purposes


  # Get content 
  for n in range(0, number_of_results, 10): 
      sleep(randint(1,5))   
      url = f'https://scholar.google.com/scholar?start={n}&q={keyword}+source:%22{source}%22+%26+source:%22{publisher}%22&hl=en&as_sdt=0,11&as_ylo={start_year}&as_yhi={end_year}&as_vis=1'
      page = session.get(url, headers={'User-Agent': 'email@gmail.com'} )
      c = page.content
      
      # Create parser
      soup = BeautifulSoup(c, 'html.parser')
      
      # Get stuff
      mydivs = soup.findAll("div", { "class" : "gs_r gs_or gs_scl" })

      
      for div in mydivs:
        try:
            links.append(div.find('h3').find('a').get('href'))
        except: # catch *all* exceptions
            links.append('Look manually at: https://scholar.google.com/scholar?start='+str(n)+'&q'+keyword.replace(' ','+'))
        
        try:
            title.append(div.find('h3').find('a').text)
        except: 
            title.append('Could not catch title')

            
            
        try:
            year.append(get_year(div.find('div',{'class' : 'gs_a'}).text))
        except: 
            year.append('Could not catch title')      


        try:
            citations.append(get_citations(str(div.format_string)))
        except: 
            citations.append('Could not catch title')            
        
        try:
            author.append(get_author(div.find('div',{'class' : 'gs_a'}).text))
        except: 
            author.append('Could not catch title')   

        try:
            jour.append(get_journal(div.find('div',{'class' : 'gs_a'}).text))
        except: 
            jour.append('Could not catch title')  

        journal.append(source)
      
        rank.append(rank[-1]+1)
      
      if len(mydivs) == 0:
        break    # break here



  # Create a dataset and sort by the number of citations
  data = pd.DataFrame(zip(author, title, citations, links, year, journal, jour), index = rank[1:], 
                      columns=['Author', 'Title', 'Citations',  'Link', 'Year', 'Journal', 'jour'])

  return data

## Search Parameters

In [25]:
keyword = "Deep Learning"
source = ""
publisher = ""  # the double quote will look for the exact keyword, # the simple quote will also look for similar keywords
number_of_results = 100 #This number restricts the number of results. 1000 is max for google scholar
start_year=2021
end_year=2022

In [26]:
data=result(keyword, source, publisher, start_year, end_year, number_of_results)

In [27]:
data.sort_values(by='Citations', ascending=False, inplace=True)
data.reset_index(drop=True, inplace=True)
data

,Author,Title,Citations,Link,Year,Journal,jour
0,Chollet,Deep learning with Python,4108,https://books.google.com/books?hl=en&lr=&id=mj...,2021,,2021
1,"Minaee, YY Boykov, F Porikli, AJ Plaza…",Image segmentation using deep learning: A survey,1182,https://ieeexplore.ieee.org/abstract/document/...,2021,,"IEEE transactions on …, 2021"
2,"Alzubaidi, J Zhang, AJ Humaidi, A A","Review of deep learning: Concepts, CNN archite...",815,https://link.springer.com/article/10.1186/s405...,2021,,"Dujaili… - Journal of big Data, 2021"
3,"Pang, C Shen, L Cao, AVD Hengel",Deep learning for anomaly detection: A review,710,https://dl.acm.org/doi/abs/10.1145/3439950,2021,,"ACM Computing Surveys (CSUR), 2021"
4,"Ye, J Shen, G Lin, T Xiang, L Shao…",Deep learning for person re-identification: A ...,661,https://ieeexplore.ieee.org/abstract/document/...,2021,,"IEEE transactions on …, 2021"
...,...,...,...,...,...,...,...
95,"Abidi, H Alkhalefah, K Moiduddin, M Alazab…",Optimal 5G network slicing using machine learn...,53,https://www.sciencedirect.com/science/article/...,2021,,"Computer Standards & …, 2021"
96,"Bhatt, I Kumar, V Vijayakumar, KU Singh…",The state of the art of deep learning models i...,50,https://link.springer.com/article/10.1007/s005...,2021,,"Multimedia Systems, 2021"
97,"Wang, L Wang, EH Lee, J Zheng, W Zhang…",Decoding COVID-19 pneumonia: comparison of dee...,43,https://link.springer.com/article/10.1007/s002...,2021,,"European journal of …, 2021"
98,"Roberts, S Yaida, B Hanin",The principles of deep learning theory,41,https://www.cambridge.org/core/services/aop-ca...,2021,,"arXiv preprint arXiv:2106.10165, 2021"
